In [1]:
import pandas as pd
#import re
#import html
import sys
#import os 

from collections import defaultdict

import pickle

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity



sys.path.append('..')


from helper_functions import *
from main_functions import *


with open('../dictionaries/dix_acad.pkl', 'rb') as f:
    dix_acad = pickle.load(f)

with open('../dictionaries/dix_mult.pkl', 'rb') as f:
    dix_mult = pickle.load(f)

with open('../dictionaries/dix_city.pkl', 'rb') as f:
    dix_city = pickle.load(f)
    
with open('../dictionaries/dix_country.pkl', 'rb') as f:
    dix_country = pickle.load(f)
    
    

def find_ror(string, simU, simG):
    print(string)
    df = pd.DataFrame()
 
    df['Unique affiliations'] = [[string]]
    academia = create_df_algorithm(df)
    
    result = Aff_Ids(len(academia), academia,dix_acad, dix_mult, dix_city, dix_country, simU, simG)
    
    if len(result)>0:
         
        
        dict_aff_open = {x: y for x, y in zip(result['Original affiliations'], result['Matched organizations'])}
        dict_aff_id = {x: y for x, y in zip(result['Original affiliations'], result['ROR'])}
    
        dict_aff_score = {}
        for i in range(len(result)):
            if type(result['Similarity score'].iloc[i]) == list:
                dict_aff_score[result['Original affiliations'].iloc[i]] = result['Similarity score'].iloc[i]
            else:
                dict_aff_score[result['Original affiliations'].iloc[i]] = [result['Similarity score'].iloc[i]]
                

        pids = []
        for i in range(len(df)):
            pidsi = []
            for aff in df['Unique affiliations'].iloc[i]:
                if aff in list(dict_aff_id.keys()):
                    pidsi = pidsi + dict_aff_id[aff]
            # elif 'unmatched organization(s)' not in pidsi:
            #     pidsi = pidsi + ['unmatched organization(s)']
            pids.append(pidsi)
                    
                    
        names = []
        for i in range(len(df)):
            namesi = []
            for aff in df['Unique affiliations'].iloc[i]:
                if aff in list(dict_aff_open.keys()):
                    try:
                        namesi = namesi + dict_aff_open[aff]
                    except TypeError:
                        namesi = namesi + [dict_aff_open[aff]]
                    
            names.append(namesi)
            
        scores = []
        for i in range(len(df)):
            scoresi = []
            for aff in df['Unique affiliations'].iloc[i]:
                if aff in list(dict_aff_score.keys()):
                    scoresi = scoresi +  dict_aff_score[aff]
                    
            scores.append(scoresi)
            
            
        df['Matched organizations'] = names
        df['ROR'] = pids
        df['Scores'] = scores


       
        def update_Z(row):
            if len(row['ROR']) == 0 or len(row['Scores']) == 0:
                return []
            
            new_Z = []
            for ror, score in zip(row['ROR'], row['Scores']):
                entry = {'ROR_ID': ror, 'Score': score}
                new_Z.append(entry)
            return new_Z

        matching = df.apply(update_Z, axis=1)

        df['Matchings'] = matching
        
        return df['Matchings'].iloc[0]
    else: 
        return print('no result')

     

default values: simU = 0.55, simlG = 0.87

In [2]:
find_ror('university of athens', 0.55, 0.87)

university of athens


[{'ROR_ID': 'https://ror.org/04gnjpq42', 'Score': 1}]